ASAP CRN Metadata validation - wave 2

# Team Jakobsson. ASAP CRN Metadata validation - wave 2

28 Nov 2023
Andy Henrie




In [1]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector, sanitize
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2

Streamlit NOT successfully


## copy metadata locally


In [2]:
!gsutil -u dnastack-asap-parkinsons cp -r "gs://asap-raw-data-team-jakobsson/ogmetadata/*.csv" "/Users/ergonyc/Projects/ASAP/team-jakobsson/metadata/"

AccessDeniedException: 403 raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).


Fix a typo, and then reupload.

```python 
"""
    Hi Andy,
    That’s a typo on my end.

    NP18-00117 should be BB18.0035.
    NP18-00287 is correct.

    CLINPATH+SUBJECT
    Cheers,

    Oliver
"""
```

In [4]:
# metadata_version = "v2"

# CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
# METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


# ## convert 
# data_path = Path.home() / ("Projects/ASAP/team-jakobsson")
# metadata_path = data_path / "metadata"


# SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
# SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
# CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
# STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)

# fix typos
# STUDY.rename({'submittor_email':'submitter_email'}, axis=1, inplace=True) #typo from CDE 

read url


In [5]:
# row = SUBJECT['subject_id'] == 'NP18-117'
# col = 'source_subject_id'
# #fix typo
# print(f"before: {SUBJECT.loc[row,col]}")
# SUBJECT.loc[row,col] = 'BB18.0035'
# print(f"after: {SUBJECT.loc[row,col]}")

# row = CLINPATH['subject_id'] == 'NP18-117'
# col = 'source_subject_id'
# #fix typo
# print(f"before: {CLINPATH.loc[row,col]}")
# CLINPATH.loc[row,col] = 'BB18.0035'
# print(f"after: {CLINPATH.loc[row,col]}")



before: 11    BB18.0035
Name: source_subject_id, dtype: string
after: 11    BB18.0035
Name: source_subject_id, dtype: string
before: 11    BB18.0035
Name: source_subject_id, dtype: string
after: 11    BB18.0035
Name: source_subject_id, dtype: string


In [6]:

# row = SAMPLE['sample_id'] == 'ASAP68 _PD_NP18-117_PUT'
# col = 'sample_id'
# #fix typo
# print(f"before: {SAMPLE.loc[row,col]}")
# SAMPLE.loc[row,col] = 'ASAP68_PD_NP18-117_PUT'
# print(f"after: {SAMPLE.loc[row,col]}")



before: Series([], Name: sample_id, dtype: string)
after: Series([], Name: sample_id, dtype: string)


In [6]:
!gsutil -u dnastack-asap-parkinsons cp -r "/Users/ergonyc/Projects/ASAP/team-jakobsson/metadata/*.csv" "gs://asap-raw-data-team-jakobsson/ogmetadata/" 

Copying file:///Users/ergonyc/Projects/ASAP/team-jakobsson/metadata/CLINPATH.csv [Content-Type=text/csv]...
AccessDeniedException: 403 raw-admin-scherzer@dnastack-asap-parkinsons.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).


In [7]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-jakobsson")
metadata_path = data_path / "metadata"
metadata_path = Path.cwd() / "clean/team-Jakobsson"


# SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
# CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
# STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
# PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
# SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
# DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv", index_col=0)
CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv", index_col=0)
STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv", index_col=0)
PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv", index_col=0)
SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv", index_col=0)
DATA = pd.read_csv(f"{metadata_path}/DATA.csv", index_col=0)


read url


In [10]:
for col in SUBJECT.columns:

subject_id
source_subject_id
AMPPD_id
GP2_id
biobank_name
organism
sex
age_at_collection
race
ethnicity
family_history
last_diagnosis
age_at_onset
age_at_diagnosis
first_motor_symptom
hx_dementia_mci
hx_melanoma
education_level
smoking_status
smoking_years
APOE_e4_status
cognitive_status
time_from_baseline
primary_diagnosis
primary_diagnosis_text


In [40]:
STUDY = prep_table(STUDY, CDE_df)

STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.rename({'submittor_email':'submitter_email'}, axis=1, inplace=True)
# STUDY['ASAP_team_name']= "TEAM-JAKOBSSON"


In [41]:

report = ReportCollector(destination="")
validate_table(STUDY, "STUDY", CDE_df, report)
report.print_log()


All required fields are present in *STUDY* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in STUDY. 🥳



In [42]:
PROTOCOL = prep_table(PROTOCOL, CDE_df)

report = ReportCollector(destination="")
validate_table(PROTOCOL, "PROTOCOL", CDE_df, report)
report.print_log()


All required fields are present in *PROTOCOL* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in PROTOCOL. 🥳



In [43]:

report = ReportCollector(destination="")
validate_table(SUBJECT, "SUBJECT", CDE_df, report)
report.print_log()


All required fields are present in *SUBJECT* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- ethnicity: 25/25 empty rows

	- age_at_onset: 25/25 empty rows

	- age_at_diagnosis: 17/25 empty rows

	- first_motor_symptom: 25/25 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- smoking_years: 25/25 empty rows

	- primary_diagnosis_text: 25/25 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*race*_:  invalid values 💩'<NA>'
    - valid ➡️ 'American Indian or Alaska Native', 'Asian', 'White', 'Black or African American', 'Multi-racial', 'Native Hawaiian or Other Pacific Islander', 'Other', 'Unknown', 'Not Reported'
- _*family_history*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No', 'Unknown', 'Not Reported'
- _*hx_dementia_mci*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*hx_melanoma*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Yes', 'No'
- _*education_level*_:  invalid values 💩'<NA>'
    - valid ➡️ 'High School', 'High School/GED', 'Some college w

In [44]:
SAMPLE = prep_table(SAMPLE, CDE_df)
# depricate source_sample_id .  These are clearly derivative of the teams subject ID rather than from source
SAMPLE['source_sample_id'] = pd.NA

In [45]:
    
report = ReportCollector(destination="")
validate_table(SAMPLE, "SAMPLE", CDE_df, report)
report.print_log()

All required fields are present in *SAMPLE* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- RIN: 71/71 empty rows

	- source_RIN: 71/71 empty rows
🚨⚠️❗ **Optional Fields with Empty (nan) values:**

	- DV200: 71/71 empty rows

	- pm_PH: 71/71 empty rows
## Enums
🚨⚠️❗ **Invalid entries**
- _*hemisphere*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Right', 'Left', 'Unknown'
- _*region_level_2*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Prefrontal cortex', 'Superior frontal gyrus', 'Middle frontal gyrus', 'Inferior frontal gyrus', 'Superior temporal gyrus', 'Middle temporal gyrus', 'Inferior temporal gyrus', 'Fusiform gyrus', 'Transentorhinal region', 'Entorinal region', 'Subiculum', 'CA1-CA4', 'Amygdala', 'Periamygdala cortex', 'Anterior cingulate gyrus', 'Posterior cingulate gyrus', 'Superior parietal lobule', 'Inferior parietal lobule', 'Parastriate cortex', 'Peristriate cortex', 'Striate cortex', 'Insular cortex', 'Caudate nucleus', 'Putamen', 'Globus pallidus', 'Thalamus', 'S

In [46]:

CLINPATH = prep_table(CLINPATH, CDE_df)



In [47]:
CLINPATH = reorder_table_to_CDE(CLINPATH, "CLINPATH", CDE_df)




In [48]:
report = ReportCollector(destination="")
validate_table(CLINPATH, "CLINPATH", CDE_df, report)
report.print_log()


All required fields are present in *CLINPATH* table.
🚨⚠️❗ **Required Fields with Empty (nan) values:**

	- duration_pmi: 7/25 empty rows

	- path_year_death: 25/25 empty rows

	- brain_weight: 25/25 empty rows
No empty entries (Nan) found in _Optional_ fields.
## Enums
🚨⚠️❗ **Invalid entries**
- _*path_braak_nft*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', 'I', 'II', 'III', 'IV', 'V', 'VI', 'I/II', 'III/IV', 'IV/V', 'V/VI'
- _*path_braak_asyn*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '6', '1/2', '3/4', '4/5', '5/6'
- _*path_cerad*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Sparse', 'Moderate', 'Frequent'
- _*path_thal*_:  invalid values 💩'<NA>'
    - valid ➡️ '0', '1', '2', '3', '4', '5', '1/2', '3', '4/5'
- _*known_pathogenic_mutation*_:  invalid values 💩'<NA>'
    - valid ➡️ 'None', 'Present', 'Unknown'
- _*path_mckeith*_:  invalid values 💩'<NA>'
    - valid ➡️ 'Neocortical', 'Limbic (transitional)', 'Brainstem', 'Amygdala Predominant', 'Olfacto

In [49]:
DATA = prep_table(DATA, CDE_df)

report = ReportCollector(destination="")
validate_table(DATA, "DATA", CDE_df, report)
report.print_log()

All required fields are present in *DATA* table.
No empty entries (Nan) found in _Required_ fields.
No empty entries (Nan) found in _Optional_ fields.
## Enum fields have valid values in DATA. 🥳



In [54]:
# STUDY.fillna("", inplace=True)
# PROTOCOL.fillna("", inplace=True)
# SAMPLE.fillna("", inplace=True)
# SUBJECT.fillna("", inplace=True)
# CLINPATH.fillna("", inplace=True)
# DATA.fillna("", inplace=True)

CLINPATH = sanitize(CLINPATH)
PROTOCOL = sanitize(PROTOCOL)
SUBJECT = sanitize(SUBJECT)
STUDY = sanitize(STUDY)
DATA = sanitize(DATA)
SAMPLE = sanitize(SAMPLE)


Streamlit NOT successfully


In [55]:

export_root = Path.cwd() / "clean/team-Jakobsson"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


In [61]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,ASAP1_PD_NP16-162_SN,1,1,0,1,fastq,ASAP1_PD_NP16-162_SN_R1.fastq.gz,Barcode & UMI sequence,cb8b16ec040ba7b7f18b446eb8135500,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
1,ASAP2_PD_NP16-25_SN,1,1,0,1,fastq,ASAP2_PD_NP16-25_SN_R1.fastq.gz,Barcode & UMI sequence,380812c09848e08dbd5035f8cf3686a8,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
2,ASAP3_PD_P73_SN,1,1,0,1,fastq,ASAP3_PD_P73_SN_R1.fastq.gz,Barcode & UMI sequence,604748473940c0e3a7441314d85b6550,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
3,ASAP4_PD_P74_SN,1,1,0,1,fastq,ASAP4_PD_P74_SN_R1.fastq.gz,Barcode & UMI sequence,86c1d5232907d35b4e629d7c769514ef,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
4,ASAP5_Ctl_NP16-161_SN,1,1,0,1,fastq,ASAP5_Ctl_NP16-161_SN_R1.fastq.gz,Barcode & UMI sequence,0d417eb4629bbddd8299de991f0e1883,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>


In [62]:
TSTSTR = "-"

test1 = "before" + TSTSTR + "after"

test1.upper().replace("-","_"), ord(TSTSTR), ord(TSTSTR.upper()), ord("-"), ord("_")

('BEFORE_AFTER', 45, 45, 45, 95)

In [56]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json  


Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [64]:
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-jakobsson/**/*.gz" > jakobsson_hexhash.log

Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]

using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [65]:
!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-jakobsson/**/*.gz" > new_manifest.log

In [66]:
# !gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json  
# !gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-jakobsson/CRN_snRNA/**/*" > manifest.txt

In [17]:

from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [18]:
bucket_files_md5 = extract_md5_from_details2("jakobsson_hexhash.log")
bucket_files_md5

{'ASAP10_PFC_count_matrix.tar.gz': '9d9825c4260f2ce9e2886151865640df',
 'ASAP11_PFC_count_matrix.tar.gz': '898b81becfa49b1a9a425c7748422ab6',
 'ASAP12_PFC_count_matrix.tar.gz': '6384e6b377571fb5d5b9407fc210bcfa',
 'ASAP13_PFC_count_matrix.tar.gz': '31ac48621143a6a347ffc960c305aa81',
 'ASAP14_PFC_count_matrix.tar.gz': 'f7a371fa2cbf4c89344cfd72638d4f67',
 'ASAP15_PFC_count_matrix.tar.gz': '1d66933bd2192ee477c5000fbbb72160',
 'ASAP16_PFC_count_matrix.tar.gz': '773cb9c9a42fc758f7f61208b0455f5f',
 'ASAP17_PFC_count_matrix.tar.gz': '8d1ca3cedf51b283c94550b6b7bdfde8',
 'ASAP18_AM_count_matrix.tar.gz': '5fa973b880bfa81b98c7ec0a12974058',
 'ASAP19_PFC_count_matrix.tar.gz': '683b5ead3e4105b7a8694664fad9899f',
 'ASAP1_SN_count_matrix.tar.gz': '7978bc4c35f3166a5ccbce94b91ca2c9',
 'ASAP20_AM_count_matrix.tar.gz': '95384e11a445690a2955ae19c20f6fdc',
 'ASAP21_SN_count_matrix.tar.gz': '53a91cba0bd2077c236bd1f05b88c959',
 'ASAP22_PFC_count_matrix.tar.gz': '728e87e88cc4af99595a0f21c55f7e1f',
 'ASAP24_AM

In [19]:



checksum = DATA[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_22072/2412817574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_22072/2412817574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [20]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [21]:
checksum[checksum.file_name.str.contains('ASAP20_PD_NP16-160_AMG_')].head()

,file_name,file_MD5,check1,check2
19,ASAP20_PD_NP16-160_AMG_R1.fastq.gz,5081d0ebf46467082ac8d8fc527fe26b,5081d0ebf46467082ac8d8fc527fe26b,5081d0ebf46467082ac8d8fc527fe26b
90,ASAP20_PD_NP16-160_AMG_R2.fastq.gz,12d3931316b607f5645412e435dd8f7f,12d3931316b607f5645412e435dd8f7f,12d3931316b607f5645412e435dd8f7f
161,ASAP20_PD_NP16-160_AMG_I1.fastq.gz,989be74189a2ecc044daa159302f545b,989be74189a2ecc044daa159302f545b,989be74189a2ecc044daa159302f545b


In [22]:
checksum[checksum.file_name.str.contains('ASAP37_PD_NP19-16_AMG_')].head()

,file_name,file_MD5,check1,check2
33,ASAP37_PD_NP19-16_AMG_R1.fastq.gz,5411b1e3626d1969c1d63517a17352a8,5411b1e3626d1969c1d63517a17352a8,5411b1e3626d1969c1d63517a17352a8
104,ASAP37_PD_NP19-16_AMG_R2.fastq.gz,315063a5cf7f889d5e9f31b4828e33ba,315063a5cf7f889d5e9f31b4828e33ba,315063a5cf7f889d5e9f31b4828e33ba
175,ASAP37_PD_NP19-16_AMG_I1.fastq.gz,0c49aa78179888dd96d5830459831554,0c49aa78179888dd96d5830459831554,0c49aa78179888dd96d5830459831554
229,ASAP37_PD_NP19-16_AMG_I2.fastq.gz,eb5e033a828ffc08051f6d0bcc503a72,eb5e033a828ffc08051f6d0bcc503a72,eb5e033a828ffc08051f6d0bcc503a72


In [23]:
check3 = _DATA[['file_name','file_MD5']]
check3[check3.file_name.str.contains('ASAP37_PD_NP19-16_AMG_')]

NameError: name '_DATA' is not defined

In [74]:
check3[check3.file_name.str.contains('ASAP20_PD_NP16-160_AMG_')]

,file_name,file_MD5
19,ASAP20_PD_NP16-160_AMG_R1.fastq.gz,5081d0ebf46467082ac8d8fc527fe26b
90,ASAP20_PD_NP16-160_AMG_R2.fastq.gz,12d3931316b607f5645412e435dd8f7f
161,ASAP20_PD_NP16-160_AMG_I1.fastq.gz,989be74189a2ecc044daa159302f545b
216,ASAP20_PD_NP16-160_AMG_I2.fastq.gz,5a358692ead8449cbcdf1aa21de4cde1


## transfer metadata Jakobsson 

In [1]:
# Jakobsson
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json 


Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]


## fix bucket file structure to match the prescribed convention

remove "/CRN_snRNA/" from path

In [24]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-jakobsson/artifacts"


                                 gs://asap-raw-data-team-jakobsson/artifacts/cellranger_counts/


In [25]:
# !gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-jakobsson/CRN_snRNA/scripts" "gs://asap-raw-data-team-jakobsson/scripts"


In [26]:
# !gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-jakobsson/CRN_snRNA/artifacts" "gs://asap-raw-data-team-jakobsson/artifacts"


In [27]:
# !gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-jakobsson/CRN_snRNA/fastqs" "gs://asap-raw-data-team-jakobsson/fastqs"


In [28]:
# !gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-jakobsson/CRN_snRNA/metadata" "gs://asap-raw-data-team-jakobsson/ogmetadata"


In [8]:
!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-jakobsson/"


        20  2023-08-15T15:13:57Z  gs://asap-raw-data-team-jakobsson/test-raw-jakobsson.txt#1692112437671820  metageneration=1
                                 gs://asap-raw-data-team-jakobsson/CRN_snRNA/
                                 gs://asap-raw-data-team-jakobsson/artifacts/
                                 gs://asap-raw-data-team-jakobsson/metadata/
                                 gs://asap-raw-data-team-jakobsson/ogmetadata/
                                 gs://asap-raw-data-team-jakobsson/scripts/
TOTAL: 1 objects, 20 bytes (20 B)


In [29]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-jakobsson/metadata/v2"

# clean up errant metadata.


      5885  2023-12-14T20:32:27Z  gs://asap-raw-data-team-jakobsson/metadata/v2/CLINPATH.csv#1702585947867836  metageneration=1
     69407  2023-12-14T20:32:28Z  gs://asap-raw-data-team-jakobsson/metadata/v2/DATA.csv#1702585948542874  metageneration=1
      1749  2023-12-14T20:32:28Z  gs://asap-raw-data-team-jakobsson/metadata/v2/PROTOCOL.csv#1702585948984510  metageneration=1
     19346  2023-12-14T20:32:28Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SAMPLE.csv#1702585948298533  metageneration=1
      2532  2023-12-14T20:32:28Z  gs://asap-raw-data-team-jakobsson/metadata/v2/STUDY.csv#1702585948739551  metageneration=1
      2040  2023-12-14T20:32:28Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SUBJECT.csv#1702585948088449  metageneration=1
                                 gs://asap-raw-data-team-jakobsson/metadata/v2/2023-11-30/
TOTAL: 6 objects, 100959 bytes (98.59 KiB)


In [57]:

!gsutil -u dnastack-asap-parkinsons cp "./clean/team-Jakobsson/*.csv"  "gs://asap-raw-data-team-jakobsson/metadata/v2/"

Copying file://./clean/team-Jakobsson/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/SUBJECT.csv [Content-Type=text/csv]...    
Copying file://./clean/team-Jakobsson/SAMPLE.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Jakobsson/DATA.csv [Content-Type=text/csv]...       
\ [4 files][ 86.2 KiB/ 86.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Jakobsson/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/PROTOCOL.csv [Content-Type=text/csv]...   
| [6 files][ 90.3 KiB/ 90.3 KiB]                                                
Operation completed over 6 objects/90.3 KiB.                                     


### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [31]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [32]:
!gsutil cp "./clean/team-Jakobsson/*.csv" "gs://asap-workflow-dev/metadata/v2/jakobsson"

Copying file://./clean/team-Jakobsson/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/SUBJECT.csv [Content-Type=text/csv]...    
Copying file://./clean/team-Jakobsson/SAMPLE.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Jakobsson/DATA.csv [Content-Type=text/csv]...       
- [4 files][ 94.4 KiB/ 94.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Jakobsson/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/PROTOCOL.csv [Content-Type=text/csv]...   
\ [6 files][ 98.6 KiB/ 98.6 KiB]                                                
Operation completed over 6 objects/98.6 KiB.                                     


In [33]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2"

                                 gs://asap-workflow-dev/metadata/v2/hafler/
                                 gs://asap-workflow-dev/metadata/v2/hardy/
                                 gs://asap-workflow-dev/metadata/v2/jakobsson/
                                 gs://asap-workflow-dev/metadata/v2/lee/
                                 gs://asap-workflow-dev/metadata/v2/scherzer/
                                 gs://asap-workflow-dev/metadata/v2/sulzer/
